# 用表单验证数据
- 即在项目目录下新建一个forms.py文件，里面定义一个Form类，继承forms或者ModelForm
- 导入：from django import forms, from django import forms.ModelForm
## 常用的Field
- 使用Field是对数据验证的第一步，你期望这个提交上来的数据是什么类型，那就使用什么类型的Field

### CharField
- 用来接收文本
- 参数
    - max_length，min_length：字段值的最大最小长度
    - required：这个字段是否是必须的。默认是必须的
    - error_messages：在某个条件验证失败的时候，自定义给出错误信息内容，它是一个字典
    
### EmailField
- 用来接收邮件，会自动验证邮件是否合法
- error_messages的key有：required，invalid
    - error_messages={"invalid":"你输入的邮件是无效的"}

### FloatField
- 用来接收浮点类型，并且如果验证通过后，会将这个字段的值转换为浮点类型
- 参数
    - max_value，min_value：最大最小的值
    - error_messages的key有：required, invalid, max_value, min_value
    
### IntegerField
- 用来接收整型，并且验证通过后，会将这个字段的值转换为整型
- 参数：
    - max_value，min_value
    - error_messages的key有：required, invalid, max_value, min_value
    
### URLField
- 用来接收url格式的字符串
- error_messages的key有：required，invalid

### 常用的验证器
- 导入：from django.core import validators
- 在验证某个字段的时候，可以传递一个validators参数用来指定验证器，进一步对数据进行过滤。验证器有很多，但是很多验证器其实已经通过这个Field或者一些参数就可以指定了，比如EmailValidator，可以通过EmailField来指定，比如MaxValueValidator，可以通过max_value参数来指定。下面是常用的验证器
    - 1.MaxValueValidator：验证最大值
    - 2.MinValueValidator:验证最小值
    - 3.MinLengthValidator:验证最小长度
    - 4.MaxLengthValidator:验证最大长度
    - 5.EmailValidator:验证是否是邮箱格式
    - 6.URLValidator:验证是否是url格式
    - 7.RegexVaildator:如果还需要更加复杂的验证，那么可以通过正则表达式的验证器，RegexValidator
- 使用示例：
    - email = forms.CharField(validators=[validators.EmailValidator(message='请输入正确的邮箱格式')])
    - telephone = forms.CharField(validators=[validators.RegexValidator(r"1[12345678]\d{9}", message='请输入正确格式的手机号码')])
    
### 自定义验证
- 有时候验证不是简单的验证器就能解决的，所以需要自定义验证。对某个字段机芯自定义的验证方式是：定义一个方法，这个方法的名义定义规则时：clean_fieldname，如果验证失败，那么就抛出一个验证错误，比如要验证用户表中手机号码之前是否在数据库中存在

# 表单制作顺序
- 1.在前端提交一个表单
- 2.服务器（即forms.py文件中）写一个form表单，对表单数据进行验证
- 3.视图函数在表单数据验证通过后，对数据做后续的处理
    - 通过form.is_valid()来调用查看forms.py文件中的数据验证是否通过，若通过就获取数据，详细代码见：django_form

# 提取错误信息
- 如果验证失败了，那么有一些错误信息是我们需要传给前端的，这时候我们可以通过以下属性来获取：
    - form.errors:这个属性获取的错误信息是个包含了HTML标签的错误信息
    - form.errors.get_json_data():这个方法获取到的是一个字典类型的错误信息。将某个字段作为key，错误信息作为值的一个字典
    - form.as_json():这个方法是将form.get_json_data()返回的字典dump成json格式的字符串，方便进行传输
    - 上述方法获取的字段的错误值，都是一个比较复杂的数据。
    - 如果只想把错误信息放在一个列表中，而不是再放在字典中，这时候我们可以定义一个方法，把这个数据重新整理一份
    
- 示例：
        class BaseForm(forms.Form):
            def get_errors(self):
                errors = self.errors.get_json_data()
                new_errors = {}
                for key, message_dicts in errors.items():
                    messages = []
                    for message_dict in message_dicts:
                        message = message_dict['message']
                        messages.append(message)
                    new_errors[key] = messages
                return new_errors
            # 直接定义个类，让其他的表单类都继承这个类，然后在输出错误信息的时候就可以直接调用这个get_errors()了，比如某个表单类print(form.get_errors())

# ModleForm
- 多数情况下表单即Form中的Field和模型中即Model的Field基本上是一模一样的，而且表单中验证的数据，基本上都是在模型中需要保存的。此时就可以将模型中的字段和表单中的字段进行绑定。而绑定就可以通过ModelForm来实现
- ModelForm是继承自forms.ModelForm，然后在表单中定义了一个Meta类，在Meta类中指定如下参数
    - 1.model=需要绑定的模型名  ，意思是该表单和哪个模型进行绑定
    - 2. field="__all__",意思是将模型中所有的字段都复制过来进行验证
        - 如果只想针对某几个字段进行验证，那么可以给Fields指定一个列表，将需要的字段写进去，比如只想验证title和content，那么field=['title', 'content'],或者就不想验证它两，exclude=['title', 'content']
    - 3.error_messages = {}:非必要参数，它是用来当验证出错时，修改错误信息提示内容的
        - 示例：
                error_messages = {
                    'page':{
                        'required': '请传入page参数',
                        'invalid': '请输入一个有效的page参数'
                        }
                }

# ModelForm中的save方法
- ModelForm还有save方法 ，可以在验证完成后直接在模型中调用save方法，就可以将这个数据保存到数据库中了
- 示例：
        def post(self, request):
            form = Myform(request.POST)
            if form.is_valid():
                form.save()
                return HttpResponse("success")

            else:
                print(form.get_errors())
                return HttpResponse("faild")
            
- 这个方法必须要在clean没有问题后才能使用，如果在clean之前使用，会抛出异常。另外，在调用save方法的时候，可以传递一个参数commit，那么只会生成这个模型的对象，而不会把这个对象真正的插入到数据库中。比如表单验证的字段没有包含模型中所有的字段，这时候就可先创建对象，再填充其他字段，把所有的字段的值都补充完成后，再保存到数据库中。
- 详情代码见djangoform